In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/MyDrive/CatClassifier/train'

In [ ]:
train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=15,)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='training',
    class_mode='binary',
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    class_mode='binary'
)

Found 20001 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
model.trainable = False

In [ ]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:
global_average_layer = keras.layers.GlobalAveragePooling2D()
prediction_layer = keras.layers.Dense(2, activation='softmax')

In [ ]:
model = keras.models.Sequential([
  model,
  global_average_layer,
  prediction_layer
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, decay = 0.0005),
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/CatClassifier/MobileNet',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto',)

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience = 5, verbose = 1, mode= 'auto')

In [ ]:
num_train_samples = train_generator.n
num_train_batches = train_generator.batch_size
train_size = num_train_samples // num_train_batches

In [ ]:
num_val_samples = validation_generator.n
num_val_batches = validation_generator.batch_size
val_size = num_val_samples // num_val_batches

In [ ]:
hist = model.fit(train_generator,
                 steps_per_epoch=train_size, 
                 epochs = 100 , 
                 validation_data=validation_generator ,
                 validation_steps=val_size ,
                 verbose=1,
                 callbacks=[checkpoint, early])

Epoch 1/100
625/625 [==============================] - 9802s 16s/step - loss: 0.1558 - accuracy: 0.9370 - val_loss: 0.0776 - val_accuracy: 0.9720

Epoch 00001: val_loss improved from inf to 0.07762, saving model to /content/drive/MyDrive/CatClassifier/MobileNet
INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/MobileNet/assets
Epoch 2/100
625/625 [==============================] - 1168s 2s/step - loss: 0.0719 - accuracy: 0.9734 - val_loss: 0.0705 - val_accuracy: 0.9716

Epoch 00002: val_loss improved from 0.07762 to 0.07054, saving model to /content/drive/MyDrive/CatClassifier/MobileNet
INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/MobileNet/assets
Epoch 3/100
625/625 [==============================] - 1163s 2s/step - loss: 0.0631 - accuracy: 0.9765 - val_loss: 0.0694 - val_accuracy: 0.9732

Epoch 00003: val_loss improved from 0.07054 to 0.06936, saving model to /content/drive/MyDrive/CatClassifier/MobileNet
INFO:tensorflow:Assets written